https://github.com/eriklindernoren/Keras-GAN
conda install scipy==1.1.0

In [1]:
import scipy
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.datasets import mnist
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
# from data_loader import DataLoader
import numpy as np
import os

In [2]:
import scipy
from glob import glob
import numpy as np

class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('./datasets/%s/%sA/*' % (self.dataset_name, data_type))
        path_B = glob('./datasets/%s/%sB/*' % (self.dataset_name, data_type))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [3]:
class CycleGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'apple2orange'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)

        # Upsampling
        u1 = deconv2d(d4, d3, self.gf*4)
        u2 = deconv2d(u1, d2, self.gf*2)
        u3 = deconv2d(u2, d1, self.gf)

        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i, self.data_loader.n_batches,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)

    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

In [4]:
if __name__ == '__main__':
    gan = CycleGAN()
    gan.train(epochs=200, batch_size=1, sample_interval=200)

C:\Users\12810649\AppData\Local\Continuum\anaconda3\envs\tf2.1-gpu\lib\site-packages\ipykernel_launcher.py:75: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
C:\Users\12810649\AppData\Local\Continuum\anaconda3\envs\tf2.1-gpu\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
C:\Users\12810649\AppData\Local\Continuum\anaconda3\envs\tf2.1-gpu\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


[Epoch 0/200] [Batch 0/995] [D loss: 2.315166, acc:  52%] [G loss: 34.211811, adv: 8.716078, recon: 0.764815, id: 0.941191] time: 0:00:15.692630 


C:\Users\12810649\AppData\Local\Continuum\anaconda3\envs\tf2.1-gpu\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


[Epoch 0/200] [Batch 1/995] [D loss: 6.553207, acc:  53%] [G loss: 42.973740, adv: 12.996815, recon: 0.772821, id: 0.910201] time: 0:00:16.135447 
[Epoch 0/200] [Batch 2/995] [D loss: 7.298995, acc:  53%] [G loss: 34.577332, adv: 9.751639, recon: 0.679060, id: 0.774277] time: 0:00:16.291988 
[Epoch 0/200] [Batch 3/995] [D loss: 10.091642, acc:  53%] [G loss: 37.154732, adv: 10.580391, recon: 0.697843, id: 1.003816] time: 0:00:16.441588 
[Epoch 0/200] [Batch 4/995] [D loss: 6.872299, acc:  51%] [G loss: 60.409325, adv: 22.900751, recon: 0.631502, id: 0.787724] time: 0:00:16.589199 
[Epoch 0/200] [Batch 5/995] [D loss: 12.102813, acc:  52%] [G loss: 46.340446, adv: 15.356465, recon: 0.675268, id: 0.951849] time: 0:00:16.740769 
[Epoch 0/200] [Batch 6/995] [D loss: 8.090006, acc:  51%] [G loss: 23.405834, adv: 4.224722, recon: 0.652012, id: 0.999709] time: 0:00:16.901073 
[Epoch 0/200] [Batch 7/995] [D loss: 3.766571, acc:  50%] [G loss: 20.765570, adv: 2.229464, recon: 0.718250, id: 1.12

[Epoch 0/200] [Batch 56/995] [D loss: 0.343157, acc:  60%] [G loss: 13.995364, adv: 0.698671, recon: 0.546851, id: 1.120688] time: 0:00:25.130015 
[Epoch 0/200] [Batch 57/995] [D loss: 0.437148, acc:  58%] [G loss: 13.317724, adv: 0.659381, recon: 0.521594, id: 0.892669] time: 0:00:25.284160 
[Epoch 0/200] [Batch 58/995] [D loss: 0.331889, acc:  59%] [G loss: 13.850561, adv: 0.502406, recon: 0.563407, id: 0.889677] time: 0:00:25.442737 
[Epoch 0/200] [Batch 59/995] [D loss: 0.377762, acc:  55%] [G loss: 13.374534, adv: 0.505915, recon: 0.516232, id: 0.976140] time: 0:00:25.600317 
[Epoch 0/200] [Batch 60/995] [D loss: 0.355026, acc:  61%] [G loss: 12.026457, adv: 0.661458, recon: 0.448000, id: 0.929507] time: 0:00:25.766699 
[Epoch 0/200] [Batch 61/995] [D loss: 0.399515, acc:  55%] [G loss: 10.384264, adv: 0.512936, recon: 0.381060, id: 1.025270] time: 0:00:25.919289 
[Epoch 0/200] [Batch 62/995] [D loss: 0.262014, acc:  68%] [G loss: 8.749576, adv: 0.698435, recon: 0.280651, id: 1.07

...

[Epoch 7/200] [Batch 325/995] [D loss: 0.144613, acc:  78%] [G loss: 5.829176, adv: 0.761061, recon: 0.167217, id: 0.543037] time: 0:20:09.141226 
[Epoch 7/200] [Batch 326/995] [D loss: 0.081187, acc:  92%] [G loss: 6.586002, adv: 0.882433, recon: 0.187687, id: 0.741805] time: 0:20:09.297057 
[Epoch 7/200] [Batch 327/995] [D loss: 0.082253, acc:  91%] [G loss: 8.049347, adv: 1.008785, recon: 0.242362, id: 0.614435] time: 0:20:09.459641 
[Epoch 7/200] [Batch 328/995] [D loss: 0.245002, acc:  74%] [G loss: 7.004569, adv: 0.928823, recon: 0.218381, id: 0.415893] time: 0:20:09.618217 
[Epoch 7/200] [Batch 329/995] [D loss: 0.117615, acc:  89%] [G loss: 7.986806, adv: 0.817327, recon: 0.251584, id: 0.800211] time: 0:20:09.776794 
[Epoch 7/200] [Batch 330/995] [D loss: 0.146448, acc:  80%] [G loss: 5.604872, adv: 0.723464, recon: 0.156371, id: 0.630429] time: 0:20:09.935951 
[Epoch 7/200] [Batch 331/995] [D loss: 0.192763, acc:  75%] [G loss: 6.911139, adv: 0.963425, recon: 0.194653, id: 0.5

[Epoch 7/200] [Batch 381/995] [D loss: 0.056303, acc:  96%] [G loss: 6.339261, adv: 0.892287, recon: 0.171575, id: 0.576650] time: 0:20:18.131461 
[Epoch 7/200] [Batch 382/995] [D loss: 0.165318, acc:  77%] [G loss: 7.629220, adv: 0.736027, recon: 0.249194, id: 0.615405] time: 0:20:18.288036 
[Epoch 7/200] [Batch 383/995] [D loss: 0.240559, acc:  58%] [G loss: 6.568598, adv: 0.881372, recon: 0.189904, id: 0.516975] time: 0:20:18.445173 
[Epoch 7/200] [Batch 384/995] [D loss: 0.121103, acc:  87%] [G loss: 6.797311, adv: 0.813042, recon: 0.206861, id: 0.435016] time: 0:20:18.608770 
[Epoch 7/200] [Batch 385/995] [D loss: 0.116106, acc:  83%] [G loss: 8.529921, adv: 1.083621, recon: 0.256142, id: 0.783838] time: 0:20:18.768340 
[Epoch 7/200] [Batch 386/995] [D loss: 0.228024, acc:  67%] [G loss: 6.201973, adv: 0.946382, recon: 0.153557, id: 0.705159] time: 0:20:18.925787 
[Epoch 7/200] [Batch 387/995] [D loss: 0.101021, acc:  91%] [G loss: 8.340903, adv: 0.833230, recon: 0.270218, id: 0.6

[Epoch 7/200] [Batch 437/995] [D loss: 0.043363, acc:  98%] [G loss: 8.262179, adv: 0.891972, recon: 0.248798, id: 0.913549] time: 0:20:27.417073 
[Epoch 7/200] [Batch 438/995] [D loss: 0.123117, acc:  86%] [G loss: 7.505713, adv: 0.787237, recon: 0.219230, id: 0.926165] time: 0:20:27.567698 
[Epoch 7/200] [Batch 439/995] [D loss: 0.078331, acc:  94%] [G loss: 7.009504, adv: 0.857745, recon: 0.203675, id: 0.696799] time: 0:20:27.720699 
[Epoch 7/200] [Batch 440/995] [D loss: 0.155646, acc:  75%] [G loss: 6.836627, adv: 0.748798, recon: 0.202244, id: 0.680532] time: 0:20:27.870312 
[Epoch 7/200] [Batch 441/995] [D loss: 0.066403, acc:  96%] [G loss: 6.720161, adv: 0.902234, recon: 0.197883, id: 0.630494] time: 0:20:28.024899 
[Epoch 7/200] [Batch 442/995] [D loss: 0.050744, acc:  98%] [G loss: 6.137307, adv: 0.932538, recon: 0.169990, id: 0.587131] time: 0:20:28.182506 
[Epoch 7/200] [Batch 443/995] [D loss: 0.073809, acc:  92%] [G loss: 8.403112, adv: 0.779746, recon: 0.255343, id: 0.8

[Epoch 7/200] [Batch 493/995] [D loss: 0.081804, acc:  93%] [G loss: 6.483413, adv: 0.922172, recon: 0.178748, id: 0.345005] time: 0:20:37.061890 
[Epoch 7/200] [Batch 494/995] [D loss: 0.091895, acc:  92%] [G loss: 8.339579, adv: 0.996465, recon: 0.229183, id: 0.992351] time: 0:20:37.226450 
[Epoch 7/200] [Batch 495/995] [D loss: 0.105006, acc:  86%] [G loss: 6.593228, adv: 0.904200, recon: 0.198913, id: 0.355382] time: 0:20:37.389016 
[Epoch 7/200] [Batch 496/995] [D loss: 0.133434, acc:  85%] [G loss: 6.743486, adv: 0.967565, recon: 0.181335, id: 0.533700] time: 0:20:37.549587 
[Epoch 7/200] [Batch 497/995] [D loss: 0.056579, acc:  97%] [G loss: 7.711860, adv: 0.936312, recon: 0.242870, id: 0.703913] time: 0:20:38.233759 
[Epoch 7/200] [Batch 498/995] [D loss: 0.093029, acc:  86%] [G loss: 7.153470, adv: 0.862246, recon: 0.226176, id: 0.537862] time: 0:20:38.394330 
[Epoch 7/200] [Batch 499/995] [D loss: 0.065657, acc:  96%] [G loss: 6.066570, adv: 0.968350, recon: 0.159140, id: 0.5

KeyboardInterrupt: 

In [19]:
# https://pythonexamples.org/python-opencv-cv2-create-video-from-images/

import os
import cv2
import numpy as np

IMG_SIZE   = 256 # 画像サイズ
BLOCK_SIZE = 64  # 黒ブロックサイズ

# 動画作成
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')
video  = cv2.VideoWriter('ImgVideo.mp4', fourcc, 100.0, (IMG_SIZE, IMG_SIZE))

# for img_file in outimg_files:
for img_file in glob.glob('datasets/apple2orange/trainB/*.jpg'):
#     print(img_file)
    img = cv2.imread(img_file)
    video.write(img)

video.release()